<a href="https://colab.research.google.com/github/ssubbinn/ESAA/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
train = pd.read_table('/content/drive/MyDrive/YB_2조/미니프로젝트/3. movies-모델링/movies_train.csv',sep=',')
test = pd.read_table('/content/drive/MyDrive/YB_2조/미니프로젝트/3. movies-모델링/movies_test.csv',sep=',')
submission = pd.read_table('/content/drive/MyDrive/YB_2조/미니프로젝트/3. movies-모델링/submission.csv',sep=',')

### 데이터 전처리

object인 title, distributor, genre, release_time, screening_rat, director 가공 필요해보임. 그리고 NaN 값이 너무 많은 dir_prev_bfnum 피처도 가공 필요.

* title : **없애기**
* distributor : 169개로 너무 많음. **10개 이상 제작해본 제작사(14개)만 뽑고 나머지는 기타 처리**
* genre : **12가지니까 인코딩**
* release_time : **datetime으로 변경**
* screening_rat : **4가지니까 인코딩**
* director : 472명으로 너무 많음. 홍상수만 너무 많고 급격히 감소. **삭제**
* dir_prev_bfnum : **Nan을 0으로 대체**

**1. release_time을 datetime 타입으로 변환 후 년, 월, 일 추출**

In [ ]:
#문자열을 datetime 타입으로 변경
train['release_time']=train.release_time.apply(pd.to_datetime)
#datetime 타입에서 년, 월, 일 추출
train['release_year']=train.release_time.apply(lambda x : x.year)
train['release_month']=train.release_time.apply(lambda x : x.month)
train['release_day']=train.release_time.apply(lambda x : x.day)

In [ ]:
#문자열을 datetime 타입으로 변경
test['release_time']=test.release_time.apply(pd.to_datetime)
#datetime 타입에서 년, 월, 일 추출
test['release_year']=test.release_time.apply(lambda x : x.year)
test['release_month']=test.release_time.apply(lambda x : x.month)
test['release_day']=test.release_time.apply(lambda x : x.day)

**2. title, director, release_time 변수 제거**

- title, director 변수는 의미없다고 판단함
- release time에서 년, 월, 일 추출했으므로 제거

In [ ]:
train.drop(['title', 'director', 'release_time'], axis=1, inplace=True)
test.drop(['title', 'director', 'release_time'], axis=1, inplace=True)

**3. dir_prev_num 의 nan을 0으로 변환**

In [ ]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

**4. distributor -10개 이상 제작해본 제작사(14개)만 뽑고 나머지는 기타 처리**

In [ ]:
distributor_list=train.distributor.value_counts().loc[lambda x:x>9]
def func(distributor):
    if distributor in distributor_list:
        return distributor
    else:
        return '기타'

train['distributor'] = train['distributor'].apply(lambda x : func(x))
test['distributor'] = test['distributor'].apply(lambda x : func(x))

**[인코딩]**
- genre (숫자 인코딩)

In [ ]:
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

- screening_rat (라벨 인코딩)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
encoder.fit(train.screening_rat)
train.screening_rat=encoder.transform(train.screening_rat)
test.screening_rat=encoder.transform(test.screening_rat)

- distributor (랭크 인코딩)

In [ ]:
nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')
nm_rank['num_rank'] = [i + 1 for i in range(nm_rank.shape[0])]
nm_rank

,distributor,num_rank
7,골든타이드픽처스,1
1,(주) 케이알씨지,2
3,(주)마운틴픽쳐스,3
12,어뮤즈,4
0,(주) 영화사조제,5
5,(주)키노아이,6
14,인디스토리,7
13,영화사 진진,8
11,시네마달,9
8,기타,10


In [ ]:
train = pd.merge(train, nm_rank, how = 'left')
test = pd.merge(test, nm_rank, how = 'left')

### 모델링

In [ ]:
X_train = train.drop(['box_off_num','distributor','genre'],axis= 1)
test=test.drop(['distributor','genre'],axis= 1)
y_train = train['box_off_num']

**하이퍼 파라미터 튜닝-GBM**---

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = GradientBoostingRegressor(random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [100,300,500], 'learning_rate' : [0.05,0.1]}

grid_cv = GridSearchCV(gbm, param_grid=params, cv=2, verbose=1, scoring='neg_mean_squared_error')
grid_cv.fit(X_train, y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)   

Fitting 2 folds for each of 6 candidates, totalling 12 fits
최적의 하이퍼 파라미터 : 
 {'learning_rate': 0.05, 'n_estimators': 100}
최고 예측 정확도 :  -2329766885361.0474


In [ ]:
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05)
gbm.fit(X_train, y_train)
gbm_pred = gbm.predict(test)

**하이퍼 파라미터 튜닝-XGBoost**

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [100,300,500], 'learning_rate' : [0.05,0.1]}
grid_cv = GridSearchCV(xgb, param_grid = params , cv = 2 ,verbose=1, scoring='neg_mean_squared_error')
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[08:16:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:16:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:16:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:16:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:16:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:16:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:16:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:16:27] WARNING: /workspace/src/objective/regres

In [ ]:
xgb = XGBRegressor(n_estimators=100, learning_rate=0.05)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(test)

[08:16:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


**하이퍼 파라미터 튜닝-LGBM**

In [ ]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [100,300,500], 'learning_rate' : [0.05,0.1]}
grid_cv = GridSearchCV(lgbm, param_grid = params , cv = 2 ,verbose=1, scoring='neg_mean_squared_error')
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
최적 하이퍼 파라미터:
 {'learning_rate': 0.05, 'n_estimators': 100}
최고 예측 정확도: -2019967275430.9292


In [ ]:
lgbm = LGBMRegressor(n_estimators=100, learning_rate=0.05)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(test)

**Final** 
- xgb 모델 채택

In [ ]:
submission['box_off_num'] = xgb_pred
submission

,title,box_off_num
0,용서는 없다,2.450389e+06
1,아빠가 여자를 좋아해,1.145504e+06
2,하모니,1.486101e+06
3,의형제,1.993756e+06
4,평행 이론,1.130360e+06
...,...,...
238,해에게서 소년에게,4.940718e+04
239,울보 권투부,7.191227e+04
240,어떤살인,2.522167e+05
241,말하지 못한 비밀,-5.490695e+03


In [ ]:
submission.to_csv('submission_smpl.csv', index = False)